# Task 1 - Apriori Algorithm for Recommender System (33 points)

**Goal:** The aim of this programming assignment is to implement the Apriori algorithm and apply it to mine frequent itemsets for recommendation. You are required to implement the algorithm from scratch by using only native Python libraries and NumPy. For efficiency you will need to convert the items to ids and sort them.

**Input:** The provided input file (`orders.txt`) contains 461 lists of items, which are orders by customers in an online retail shop. [1] Each line in the file corresponds to an order and represents a list of products a costumer bought. An example:

*Alarm Clock Bakelite Green;Panda And Bunnies Sticker Sheet*

In the example above, a customer ordered the products "Alarm Clock Bakelite Green" and "Panda And Bunnies Sticker Sheet".

**Output:** Implement the Apriori algorithm and use it to mine frequent itemsets. Set the relative minimum support to 0.025 and run the algorithm on the 461 orders of retail shop products. In other words, you need to extract all the itemsets that have an absolute support larger or equal to 12.

[1] The dataset is a modified version of the france dataset from pycaret. (https://github.com/pycaret/pycaret/blob/master/datasets/france.csv)

In [12]:
# TODO: uncomment the packages you used, please do not import additional non-native packages
# You may change the imports to the following format: from [package] import [class, method, etc.]

import collections
import itertools
import numpy as np

## 1.1 Loading the data and preprocessing (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [13]:
# TODO: read the data from the input file /data/orders.txt (1 points)

import numpy as np


with open('orders.txt', 'r') as file:
    orders = file.readlines()

# Preprocess the data
orders = [order.strip().split(';') for order in orders]

# Print the first 5 orders to check the data
print(orders[:5])


[['Alarm Clock Bakelite Pink', 'Alarm Clock Bakelite Red', 'Alarm Clock Bakelite Green', 'Panda And Bunnies Sticker Sheet', 'Stars Gift Tape', 'Inflatable Political Globe', 'Vintage Heads And Tails Card Game', 'Set/2 Red Retrospot Tea Towels', 'Round Snack Boxes Set Of4 Woodland', 'Spaceboy Lunch Box', 'Lunch Box I Love London', 'Circus Parade Lunch Box', 'Charlotte Bag Dolly Girl Design', 'Red Toadstool Led Night Light', 'Set 2 Tea Towels I Love London', 'Vintage Seaside Jigsaw Puzzles', 'Mini Jigsaw Circus Parade', 'Mini Jigsaw Spaceboy', 'Mini Paint Set Vintage', 'Greeting Card Christmas'], ['Picture Dominoes', 'Mini Jigsaw Spaceboy', 'Mini Jigsaw Dolly Girl', 'Charlotte Bag Dolly Girl Design', 'Vintage Heads And Tails Card Game', 'Polkadot Rain Hat', 'Greeting Card Birthday'], ['Edwardian Parasol Black', 'Edwardian Parasol Pink', 'Edwardian Parasol Red', 'Red Harmonica In Box', 'Lunch Box I Love London', 'Lunch Box With Cutlery Retrospot', 'Set Of Salt And Pepper Toadstools', 'Tea 

In [14]:
# TODO: determine the unique items and map the item to ids using enumerate (1 points)

# orders is our list of lists where each list is an order
items = [item for sublist in orders for item in sublist]

# Determine the unique items
unique_items = list(set(items))

# Map the unique items to ids
item_to_id = {item: id for id, item in enumerate(unique_items)}

# Mapping from id to item 
id_to_item = {id: item for id, item in enumerate(unique_items)}

print("Item to ID:", item_to_id)
print("ID to Item:", id_to_item)


Item to ID: {'Heart Of Wicker Small': 0, 'Set Of 2 Round Tins Camembert': 1, 'Red Polkadot Coffee Mug': 2, 'Lantern Cream Gazebo': 3, 'Big Doughnut Fridge Magnets': 4, 'Travel Card Wallet Transport': 5, 'Set/4 Skull Badges': 6, 'Cast Iron Hook Garden Trowel': 7, 'Kings Choice Mug': 8, 'Pack Of 6 Handbag Gift Boxes': 9, 'Ivory Pillar Candle Gold Flock': 10, 'Blue Rose Fabric Mirror': 11, 'Funky Diva Pen': 12, 'Wood Black Board Ant White Finish': 13, 'Doormat Home Sweet Home Blue': 14, 'Pig Keyring With Light & Sound': 15, 'Set 2 Pantry Design Tea Towels': 16, 'Jumbo Bag Scandinavian Blue Paisley': 17, 'Retro Coffee Mugs Assorted': 18, 'Vintage Heads And Tails Card Game': 19, 'Set/6 Fruit Salad Paper Plates': 20, 'Doormat Peace On Earth Blue': 21, 'Skull Lunch Box With Cutlery': 22, 'Wicker Wreath Small': 23, 'Tea Time Party Bunting': 24, '36 Pencils Tube Skulls': 25, 'Travel Card Wallet Vintage Ticket': 26, 'Retrospot Party Bag + Sticker Set': 27, 'Grand Chocolatecandle': 28, 'Set Of 6 

In [15]:
# TODO: map the items of the records to ids and sort each record (1 points)


# Map the items of the records to ids and sort each record
mapped_records = [sorted([item_to_id[item] for item in order]) for order in orders]


# Print the first 5 mapped and sorted records to check the data
print('Mapped Record:', mapped_records[:5])

Mapped Record: [[19, 71, 120, 351, 387, 500, 586, 682, 757, 836, 854, 936, 1018, 1124, 1140, 1429, 1450, 1479, 1505, 1525], [19, 120, 279, 351, 934, 1292, 1548], [4, 53, 67, 229, 255, 288, 480, 630, 645, 734, 906, 936, 1312, 1378, 1412, 1484], [39, 42, 60, 67, 86, 137, 212, 237, 313, 324, 330, 338, 347, 349, 353, 446, 450, 480, 519, 537, 549, 558, 586, 596, 611, 617, 640, 645, 655, 688, 700, 732, 738, 747, 750, 757, 836, 854, 865, 906, 922, 934, 943, 948, 966, 997, 1002, 1045, 1086, 1087, 1089, 1143, 1261, 1304, 1324, 1372, 1378, 1484, 1497, 1509, 1548, 1553, 1557], [4, 6, 51, 170, 237, 245, 255, 278, 302, 375, 435, 480, 502, 555, 558, 634, 640, 647, 737, 784, 854, 857, 906, 951, 989, 1012, 1018, 1092, 1116, 1179, 1214, 1235, 1288, 1290, 1298, 1333, 1375, 1389, 1492, 1548, 1557]]


## 1.2 Apriori algorithm (21 points)
### A) Prune the infrequent items (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [16]:
# TODO: calculate the support of length-1 itemsets using Counter or defaultdict (1 points)
from collections import Counter

# Flatten the list of lists and count the occurrence of each item
item_counts = Counter([item for sublist in mapped_records for item in sublist])

# Calculate the support of length-1 itemsets
l1_items = {item: count for item, count in item_counts.items() if count >= 12}

print('Support of length-1 itemsets:',l1_items)


Support of length-1 itemsets: {120: 15, 351: 26, 387: 13, 586: 37, 757: 39, 836: 40, 854: 65, 936: 27, 1018: 72, 1140: 81, 1429: 27, 1450: 50, 1479: 18, 1505: 41, 1548: 86, 53: 18, 67: 47, 255: 37, 480: 57, 630: 13, 645: 27, 734: 24, 906: 17, 1312: 27, 1378: 55, 1412: 70, 1484: 61, 137: 13, 237: 48, 313: 21, 349: 55, 353: 12, 558: 23, 640: 68, 738: 17, 1002: 54, 1045: 18, 1553: 22, 1557: 30, 170: 13, 278: 19, 375: 28, 435: 14, 555: 29, 784: 21, 857: 20, 1116: 28, 1179: 16, 1235: 19, 1288: 14, 1290: 40, 108: 26, 341: 17, 699: 75, 715: 17, 745: 13, 996: 12, 1254: 13, 1303: 48, 1138: 40, 862: 13, 684: 22, 27: 24, 124: 13, 232: 26, 357: 19, 373: 20, 598: 50, 762: 15, 788: 17, 789: 26, 945: 12, 982: 54, 1058: 24, 1064: 12, 1100: 13, 22: 20, 35: 25, 1096: 28, 1263: 15, 1481: 68, 100: 17, 281: 20, 536: 12, 911: 34, 1110: 12, 81: 13, 296: 18, 466: 23, 807: 16, 883: 28, 947: 15, 1219: 52, 1240: 32, 1299: 28, 1373: 17, 1465: 25, 372: 19, 1410: 12, 1436: 44, 1448: 32, 798: 12, 1549: 15, 282: 38, 

In [17]:
# TODO: filter out the frequent length-1 itemsets with their support (1 point)


# Filter out the frequent length-1 itemsets with their support
frequent_l1_items = {item: support for item, support in l1_items.items() if support >= 12}

# Convert the itemsets to tuples to use them as keys
frequent_itemsets = {(item,): support for item, support in frequent_l1_items.items()}

print("Frequent Length-1 Itemsets:", frequent_l1_items)
print("Frequent Itemsets:", frequent_itemsets)


Frequent Length-1 Itemsets: {120: 15, 351: 26, 387: 13, 586: 37, 757: 39, 836: 40, 854: 65, 936: 27, 1018: 72, 1140: 81, 1429: 27, 1450: 50, 1479: 18, 1505: 41, 1548: 86, 53: 18, 67: 47, 255: 37, 480: 57, 630: 13, 645: 27, 734: 24, 906: 17, 1312: 27, 1378: 55, 1412: 70, 1484: 61, 137: 13, 237: 48, 313: 21, 349: 55, 353: 12, 558: 23, 640: 68, 738: 17, 1002: 54, 1045: 18, 1553: 22, 1557: 30, 170: 13, 278: 19, 375: 28, 435: 14, 555: 29, 784: 21, 857: 20, 1116: 28, 1179: 16, 1235: 19, 1288: 14, 1290: 40, 108: 26, 341: 17, 699: 75, 715: 17, 745: 13, 996: 12, 1254: 13, 1303: 48, 1138: 40, 862: 13, 684: 22, 27: 24, 124: 13, 232: 26, 357: 19, 373: 20, 598: 50, 762: 15, 788: 17, 789: 26, 945: 12, 982: 54, 1058: 24, 1064: 12, 1100: 13, 22: 20, 35: 25, 1096: 28, 1263: 15, 1481: 68, 100: 17, 281: 20, 536: 12, 911: 34, 1110: 12, 81: 13, 296: 18, 466: 23, 807: 16, 883: 28, 947: 15, 1219: 52, 1240: 32, 1299: 28, 1373: 17, 1465: 25, 372: 19, 1410: 12, 1436: 44, 1448: 32, 798: 12, 1549: 15, 282: 38, 41

### B) Determine the frequent n itemsets (15 points)
**Task:** Solve the tasks explained in the TODOs and comments.


In [18]:
# TODO: implement the apriori_gen algorithm based on the lecture slides
def apriori_gen(itemsets):
    # Generate candidates
    candidates = set()
    for itemset1 in itemsets:
        for itemset2 in itemsets:
            union = itemset1.union(itemset2)
            if len(union) == len(itemset1) + 1:
                candidates.add(union)

    # Prune the candidates
    next_itemsets = set()
    for candidate in candidates:
        subsets = [frozenset(subset) for subset in itertools.combinations(candidate, len(candidate) - 1)]
        if all(subset in itemsets for subset in subsets):
            next_itemsets.add(candidate)

    return next_itemsets


In [19]:
# TODO: implement an algorithm to calculate the support of the given itemset (2 points)
# You do not need to implement a Hash Tree for calculating the supports.
def calculate_support(itemset, orders):
    # Count the occurrence of the itemset in the orders
    count = sum(all(item in order for item in itemset) for order in orders)

    # Calculate the support of the itemset
    support = count / len(orders)

    return support


In [20]:
# TODO: set the initial frequent itemsets which needs to be used in the first iteration (1 point)
# (It will be updated after each iteration.)
frequent_n_itemsets = None

# TODO: set the correct loop condition until the Apriori algorithm should run (1 point)
while 0:
    candidates = apriori_gen(frequent_n_itemsets)
    supports = map(calculate_support, candidates)

    # TODO: filter out the frequent candidates (2 point)
    frequent_candidates = {}

    # TODO: add the frequent candidates to frequent_itemsets (1 point)
    pass

    # replace the frequent_n_itemsets for the next iteration
    frequent_n_itemsets = [itemset for itemset in frequent_candidates]

### C) Save your results (3 points)

**Task:** Save all the frequent itemsets along with their absolute supports into a text file named `patterns.txt` and place it in the root of your zip file. Every line corresponds to exactly one frequent itemset and should be in the following format:

*support:product1;product2;product3;...*

For example, suppose an itemset (Mini Paint Set Vintage;Picture Dominoes) has an absolute support 46, then the line corresponding to this frequent itemset in `patterns.txt` should be:

*46:Mini Paint Set Vintage;Picture Dominoes*

In [22]:
def save_frequent_itemsets(itemsets, filename):
    with open(filename, 'w') as f:
        for itemset in itemsets:
            support = itemset['support']
            products = ';'.join(itemset['products'])
            f.write(f'{support}:{products}\n')

# Example usage:
itemsets = [
    {'support': 46, 'products': ['Mini Paint Set Vintage', 'Picture Dominoes']},
    
]

save_frequent_itemsets(itemsets, 'patterns.txt')


## 1.3 Recommendation (9 points)

**Task:** Imagine you should recommend 2 other products to a customer who added "Pack Of 6 Skull Paper Cups" and "Pack Of 20 Skull Paper Napkins" to the cart. Based on the results of the Apriori algorithm, implement an algorithm that returns 2 products to display to the customer on the website by maximizing the confidence that the customer will buy the product. (6 points)

**Report:** Explain your method (comments in code or summary) and display your recommendations with the confidence scores. (3 points)


In [26]:
def recommend_products(basket, frequent_itemsets, num_recommendations):
    # Finding all itemsets that contain the items in the basket
    candidates = {itemset: support for itemset, support in frequent_itemsets.items() if set(basket).issubset(set(itemset))}

    # Calculating the confidence for each itemset
    confidences = {itemset: support / frequent_itemsets[tuple(sorted(set(itemset).difference(set(basket))))] for itemset, support in candidates.items()}

    # Sorting the itemsets by confidence in descending order
    sorted_confidences = sorted(confidences.items(), key=lambda item: item[1], reverse=True)

    # Getting the top num_recommendations itemsets
    top_itemsets = sorted_confidences[:num_recommendations]

    # Getting the recommended products
    recommended_products = [set(itemset).difference(set(basket)) for itemset, confidence in top_itemsets]

    return recommended_products, top_itemsets


In [27]:
basket = ["Pack Of 6 Skull Paper Cups", "Pack Of 20 Skull Paper Napkins"]
num_recommendations = 2

recommended_products, top_itemsets = recommend_products(basket, frequent_itemsets, num_recommendations)

print("Recommended Products:", recommended_products)
print("Confidence Scores:", [confidence for itemset, confidence in top_itemsets])


Recommended Products: []
Confidence Scores: []
